# Learn Notebook

In [1]:
# Setup env
import datetime, rdflib, pdb, hashlib, requests
import os, sys, rdflib, uuid, tempfile
from bplustree import BPlusTree, StrSerializer
PROJECT_DIR = os.path.split(os.path.abspath(os.curdir))[0]
sys.path.append(PROJECT_DIR)
BASE_DIR = os.path.split(PROJECT_DIR)[0]
CACHE_DIR = os.path.join(PROJECT_DIR, "cache")
sys.path.append(CACHE_DIR)
import btree
print(sys.platform, os.path.os.name, BASE_DIR, PROJECT_DIR)

win32 nt D:\2018 D:\2018\linked-data-fragments


In [3]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt